<a href="https://colab.research.google.com/github/rakeshpsnvnk/colab-test/blob/main/Access_Token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Run this in your first Colab cell
!pip install kiteconnect

In [20]:
from kiteconnect import KiteConnect
import logging

# Optional: Enable debug logging to see what's happening
logging.basicConfig(level=logging.DEBUG)

In [21]:
# Replace with your actual credentials
API_KEY = "m43a67o2h3nk88qb"
API_SECRET = "8573kf21ykhkh2ozk99qux7hmpmpuz0v"

# Initialize KiteConnect
kite = KiteConnect(api_key=API_KEY)

In [22]:
# Generate the login URL
login_url = kite.login_url()
print("Please visit this URL to authorize the app:")
print(login_url)

Please visit this URL to authorize the app:
https://kite.zerodha.com/connect/login?api_key=m43a67o2h3nk88qb&v=3


In [23]:
# Paste the request token you got from the redirect URL
request_token = "e8inbSb9vS9jlhMFCyDqIzET5Gmq1CEh"

# Generate access token
data = kite.generate_session(request_token, api_secret=API_SECRET)
access_token = data["access_token"]

print(f"Access Token: {access_token}")

Access Token: ZyYPJTvUXjOV7MyjWjw0HbmJIOLleBsA


In [24]:
# Set the access token
kite.set_access_token(access_token)

# Test the connection
try:
    profile = kite.profile()
    print("Connection successful!")
    print(f"User ID: {profile['user_id']}")
    print(f"User Name: {profile['user_name']}")
    print(f"Email: {profile['email']}")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!
User ID: QU0244
User Name: Sri Nagavenkatanarasimha Krishna Rakeshkumar Penugonda
Email: rakesh.penugonda@gmail.com


In [25]:
try:
    holdings = kite.holdings()

    if holdings:
        import pandas as pd
        df = pd.DataFrame(holdings)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        display(df)
    else:
        print("✅ No holdings found.")
except Exception as e:
    print(f"❌ Failed to fetch holdings: {e}")


✅ No holdings found.


In [26]:
try:
    order_id = kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=kite.EXCHANGE_NSE,
        tradingsymbol="TCS",
        transaction_type=kite.TRANSACTION_TYPE_BUY,
        quantity=1,
        product=kite.PRODUCT_CNC,
        order_type=kite.ORDER_TYPE_LIMIT,
        price=3300
        )
    print("✅ Order placed successfully. Order ID:", order_id)
except Exception as e:
    print("❌ Failed to place order:", e)


✅ Order placed successfully. Order ID: 250710600498913


In [28]:
# NSE F&O OHLC Data Retrieval in Google Colab - FIXED VERSION

# RECOMMENDED: Simple and Reliable Method using yfinance
!pip install yfinance pandas numpy

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def get_current_ohlc_simple(symbol):
    """
    Simple function to get current OHLC data
    Works reliably for NSE stocks
    """
    try:
        # Ensure symbol has .NS suffix for NSE
        if not symbol.endswith('.NS'):
            symbol = symbol + '.NS'

        ticker = yf.Ticker(symbol)

        # Get last 2 days of data to ensure we have latest
        data = ticker.history(period="2d", interval="1d")

        if not data.empty:
            latest = data.iloc[-1]  # Get latest available data

            result = {
                'Symbol': symbol,
                'Date': data.index[-1].strftime('%Y-%m-%d'),
                'Open': round(latest['Open'], 2),
                'High': round(latest['High'], 2),
                'Low': round(latest['Low'], 2),
                'Close': round(latest['Close'], 2),
                'Volume': int(latest['Volume']),
                'Status': 'Success'
            }
            return result
        else:
            return {'Symbol': symbol, 'Status': 'No data available'}

    except Exception as e:
        return {'Symbol': symbol, 'Status': f'Error: {str(e)}'}

# QUICK TEST - Run this first
print("=== QUICK TEST - Current OHLC Data ===")
test_stocks = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK']

for stock in test_stocks:
    result = get_current_ohlc_simple(stock)
    if result['Status'] == 'Success':
        print(f"{result['Symbol']}: O={result['Open']}, H={result['High']}, L={result['Low']}, C={result['Close']}")
    else:
        print(f"{stock}: {result['Status']}")

print("\n" + "="*60)

def get_nse_fo_data_yfinance(symbol, period="1d"):
    """
    Get NSE F&O data using yfinance
    symbol: Stock symbol (e.g., 'RELIANCE.NS', 'TCS.NS')
    period: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
    """
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period=period)

        if not data.empty:
            # Get current/latest OHLC
            latest = data.iloc[-1]
            ohlc = {
                'Symbol': symbol,
                'Date': data.index[-1].strftime('%Y-%m-%d'),
                'Open': latest['Open'],
                'High': latest['High'],
                'Low': latest['Low'],
                'Close': latest['Close'],
                'Volume': latest['Volume']
            }
            return ohlc
        else:
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Method 2: Using nsepy (NSE-specific library) - Fixed Version
!pip install nsepy

from nsepy import get_history
from datetime import date

def get_nse_fo_data_nsepy(symbol, start_date=None, end_date=None):
    """
    Get NSE equity data using nsepy (not F&O specific)
    symbol: Stock symbol (e.g., 'RELIANCE', 'TCS')
    """
    try:
        if start_date is None:
            start_date = date.today() - timedelta(days=10)
        if end_date is None:
            end_date = date.today() - timedelta(days=1)  # Yesterday's data

        # Get equity data (futures require expiry date)
        data = get_history(symbol=symbol,
                          start=start_date,
                          end=end_date)

        if not data.empty:
            latest = data.iloc[-1]
            ohlc = {
                'Symbol': symbol,
                'Date': data.index[-1].strftime('%Y-%m-%d'),
                'Open': latest['Open'],
                'High': latest['High'],
                'Low': latest['Low'],
                'Close': latest['Close'],
                'Volume': latest['Volume']
            }
            return ohlc
        else:
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Method 2b: Get F&O data with expiry date
def get_nse_futures_data_nsepy(symbol, expiry_date):
    """
    Get NSE Futures data with specific expiry date
    symbol: Stock symbol (e.g., 'RELIANCE', 'TCS')
    expiry_date: Expiry date as datetime.date object
    """
    try:
        start_date = date.today() - timedelta(days=10)
        end_date = date.today() - timedelta(days=1)

        # Get futures data with expiry date
        data = get_history(symbol=symbol,
                          start=start_date,
                          end=end_date,
                          futures=True,
                          expiry_date=expiry_date)

        if not data.empty:
            latest = data.iloc[-1]
            ohlc = {
                'Symbol': f"{symbol}_FUT",
                'Date': data.index[-1].strftime('%Y-%m-%d'),
                'Open': latest['Open'],
                'High': latest['High'],
                'Low': latest['Low'],
                'Close': latest['Close'],
                'Volume': latest['Volume'],
                'Expiry': expiry_date.strftime('%Y-%m-%d')
            }
            return ohlc
        else:
            return None
    except Exception as e:
        print(f"Error fetching futures data for {symbol}: {e}")
        return None

# Method 3: Using requests to fetch from NSE API directly - Fixed Version
import requests
import json
import time

def get_nse_fo_data_direct(symbol):
    """
    Direct NSE API call with proper session handling
    """
    try:
        session = requests.Session()

        # Set comprehensive headers to mimic browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': '*/*',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Referer': 'https://www.nseindia.com/',
            'X-Requested-With': 'XMLHttpRequest'
        }

        session.headers.update(headers)

        # First, visit the main page to get cookies
        session.get('https://www.nseindia.com/', timeout=10)
        time.sleep(1)  # Small delay

        # Now make the API call
        url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"
        response = session.get(url, timeout=10)

        if response.status_code == 200:
            data = response.json()

            # Extract OHLC data
            price_info = data.get('priceInfo', {})
            ohlc = {
                'Symbol': symbol,
                'Open': price_info.get('open'),
                'High': price_info.get('intraDayHighLow', {}).get('max'),
                'Low': price_info.get('intraDayHighLow', {}).get('min'),
                'Close': price_info.get('close'),
                'LastPrice': price_info.get('lastPrice'),
                'Volume': data.get('securityWiseDP', {}).get('quantityTraded')
            }
            return ohlc
        else:
            print(f"API request failed with status: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Example Usage
if __name__ == "__main__":
    # List of NSE F&O stocks
    nse_fo_stocks = [
        'RELIANCE.NS',
        'TCS.NS',
        'INFY.NS',
        'HDFCBANK.NS',
        'ICICIBANK.NS',
        'SBIN.NS',
        'BHARTIARTL.NS',
        'ITC.NS',
        'HINDUNILVR.NS',
        'LT.NS'
    ]

    print("Method 1: Using yfinance")
    print("=" * 50)

    results = []
    for stock in nse_fo_stocks:
        data = get_nse_fo_data_yfinance(stock)
        if data:
            results.append(data)
            print(f"{data['Symbol']}: O={data['Open']:.2f}, H={data['High']:.2f}, L={data['Low']:.2f}, C={data['Close']:.2f}")

    # Convert to DataFrame for better display
    df = pd.DataFrame(results)
    print("\nDataFrame format:")
    print(df)

    print("\n" + "=" * 50)
    print("Method 2: Using nsepy (for equity data)")
    print("=" * 50)

    # For nsepy, use symbol without .NS suffix
    nse_symbols = ['RELIANCE', 'TCS', 'INFY', 'HDFCBANK', 'ICICIBANK']

    for symbol in nse_symbols:
        data = get_nse_fo_data_nsepy(symbol)
        if data:
            print(f"{data['Symbol']}: O={data['Open']:.2f}, H={data['High']:.2f}, L={data['Low']:.2f}, C={data['Close']:.2f}")

    print("\n" + "=" * 50)
    print("Method 2b: Futures data (requires expiry date)")
    print("=" * 50)

    # Example for futures with expiry date
    from datetime import date
    # Get next Thursday (typical F&O expiry)
    next_expiry = date(2024, 12, 26)  # Example expiry date - update as needed

    for symbol in ['RELIANCE', 'TCS']:
        data = get_nse_futures_data_nsepy(symbol, next_expiry)
        if data:
            print(f"{data['Symbol']}: O={data['Open']:.2f}, H={data['High']:.2f}, L={data['Low']:.2f}, C={data['Close']:.2f}, Expiry={data['Expiry']}")

    print("\n" + "=" * 50)
    print("Method 3: Direct NSE API (with session handling)")
    print("=" * 50)

    # For direct API, use symbol without .NS suffix
    for symbol in ['RELIANCE', 'TCS', 'INFY']:
        data = get_nse_fo_data_direct(symbol)
        if data and data['Open'] is not None:
            print(f"{data['Symbol']}: O={data['Open']}, H={data['High']}, L={data['Low']}, C={data['Close']}")
        time.sleep(2)  # Add delay between requests

# Additional utility functions
def get_multiple_stocks_ohlc(symbols, method='yfinance'):
    """
    Get OHLC data for multiple stocks
    """
    results = []

    for symbol in symbols:
        if method == 'yfinance':
            data = get_nse_fo_data_yfinance(symbol)
        elif method == 'nsepy':
            data = get_nse_fo_data_nsepy(symbol.replace('.NS', ''))
        elif method == 'direct':
            data = get_nse_fo_data_direct(symbol.replace('.NS', ''))

        if data:
            results.append(data)

    return pd.DataFrame(results)

def live_price_monitor(symbols, interval=30):
    """
    Monitor live prices (run in loop)
    interval: seconds between updates
    """
    import time

    while True:
        print(f"\n--- Live Prices at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ---")

        for symbol in symbols:
            data = get_nse_fo_data_yfinance(symbol)
            if data:
                print(f"{symbol}: {data['Close']:.2f}")

        time.sleep(interval)

# Example: Monitor specific stocks
# live_price_monitor(['RELIANCE.NS', 'TCS.NS'], interval=60)  # Update every minute

=== QUICK TEST - Current OHLC Data ===
RELIANCE.NS: O=1519.7, H=1524.7, L=1507.6, C=1508.2
TCS.NS: O=3380.0, H=3397.0, L=3356.0, C=3370.5
INFY.NS: O=1638.9, H=1638.9, L=1614.0, C=1615.1
HDFCBANK.NS: O=2016.0, H=2017.0, L=2000.0, C=2007.8
ICICIBANK.NS: O=1436.8, H=1439.0, L=1424.3, C=1425.0

Method 1: Using yfinance
RELIANCE.NS: O=1519.70, H=1524.70, L=1507.60, C=1508.20
TCS.NS: O=3380.00, H=3397.00, L=3356.00, C=3370.60
INFY.NS: O=1638.90, H=1638.90, L=1614.00, C=1615.60
HDFCBANK.NS: O=2016.00, H=2017.00, L=2000.00, C=2007.80
ICICIBANK.NS: O=1436.80, H=1439.00, L=1424.30, C=1424.90
SBIN.NS: O=811.30, H=815.50, L=807.50, C=808.70
BHARTIARTL.NS: O=2028.60, H=2028.60, L=1982.10, C=1983.30
ITC.NS: O=420.30, H=420.30, L=416.40, C=416.75
HINDUNILVR.NS: O=2423.40, H=2434.00, L=2412.10, C=2412.80
LT.NS: O=3575.00, H=3589.90, L=3573.10, C=3581.60

DataFrame format:
          Symbol        Date         Open         High          Low  \
0    RELIANCE.NS  2025-07-10  1519.699951  1524.699951  1507